# Producing the Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
sys.path

['',
 '/home/gridgway/anaconda3/lib/python36.zip',
 '/home/gridgway/anaconda3/lib/python3.6',
 '/home/gridgway/anaconda3/lib/python3.6/lib-dynload',
 '/home/gridgway/anaconda3/lib/python3.6/site-packages',
 '/home/gridgway/anaconda3/lib/python3.6/site-packages/aeosa',
 '/home/gridgway/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/gridgway/.ipython',
 '..']

## Notebook Initialization

In [ ]:
%autoreload
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from darkhistory.spec.spectrum import Spectrum
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
import darkhistory.physics as phys
import darkhistory.utilities as utils

from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator

# np.set_printoptions(threshold=np.nan)

## Import Raw Data

In [ ]:
# For complete_coarse
# xes_str = [
#     '4.540E-05', '2.746E-04'
# ]

xes_str = [
    '4.540E-05', '2.746E-04', '1.659E-03', '9.952E-03', 
    '2.660E-02', '5.732E-02', '1.192E-01', '1.680E-01', '2.142E-01', '2.689E-01', '3.823E-01', 
    '5.000E-01', '5.987E-01', '6.900E-01', '8.022E-01', 
    '9.309E-01', '9.879E-01', '9.980E-01', '9.997E-01'   
]

# xes_str = [
#     '4.540E-05', '2.746E-04', '1.659E-03', '9.952E-03', 
#     '5.732E-02', '1.680E-01', '2.689E-01', '3.823E-01', 
#     '5.000E-01', '5.987E-01', '6.900E-01', '8.022E-01', 
#     '9.309E-01', '9.879E-01', '9.980E-01', '9.997E-01'   
# ]

# xes_str = ['9.900E-01', '9.990E-01']
# xes_str = ['0.000E+00']

# file_names = [
#     '/Users/hongwan/Dropbox (MIT)/Photon Deposition/'
#     +'transferfunction_injE_complete_z_30_xe_2_pts/'
#     +'transferfunction_withloweng_cmbloss_60eV_fixed_binning_injE_complete_z_30_xe_'
#     +str+'_nointerp.p'
#     for str in xes_str    
# ]

direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/transferfunction_complete/'
direc = '/home/gridgway/Desktop/DH_Data/'
direc = '/home/gridgway/Desktop/DH_Data/tfs_xHe0/'

file_names = [
    direc
    +'transferfunction_withloweng_cmbloss_60eV_fixed_binning_complete_xe_'
    +str+'_nointerp.p'
    for str in xes_str    
]

raw_tf_tuple = [pickle.load(open(file_name, "rb")) for file_name in file_names]

# xe, in_eng, rs, eng, type={highengphot, lowengphot, lowengelec}
raw_tf_list = np.array([tup[0] for tup in raw_tf_tuple])
# Truncate the energy binning, because in_eng does not go up to the end.
# raw_tf_list = raw_tf_list[:,:,:,:494,:]
# xe, in_eng, rs
CMB_engloss_list = np.array([tup[1] for tup in raw_tf_tuple])
# xe, in_eng, rs, {H-ion, Exc, Heating, Cont.}
highdeposited_list = np.array([tup[2] for tup in raw_tf_tuple])
# xe, in_eng, rs, type={mwimp, rs}
info_list = np.array([tup[3] for tup in raw_tf_tuple])
# xe, in_eng, rs, type={photeng, eleceng}, eng
abscissae_list = np.array([tup[4] for tup in raw_tf_tuple])
# abscissae_list = abscissae_list[:,:,:,:,:494]



In [ ]:
# This checks that all of the abscissae are equal.
for i in np.arange(info_list.shape[0]):
    for j in np.arange(info_list.shape[1]):
        if not np.array_equal(
            info_list[i,j,:,1],
            info_list[0,0,:,1]
        ):
            print('NOPE!')
        for k in np.arange(info_list.shape[2]):
            if not np.array_equal(
                info_list[i,:,k,0],
                info_list[0,:,0,0]
            ):
                print('NOPE!')
            if not np.array_equal(
                abscissae_list[i,j,k,0,:],
                abscissae_list[0,0,0,0,:]
            ):
                print('NOPE!')
            if not np.array_equal(
                abscissae_list[i,j,k,1,:],
                abscissae_list[0,0,0,1,:]
            ):
                print('NOPE!')
                
# Given that they are, let's just define photeng and eleceng. 
# xe_arr = 0.5 + 0.5*np.tanh(
#     np.array(
#         [-5., -4.1]
#     )
# )

# xe_arr = 0.5 + 0.5*np.tanh(
#     [-5., -4.1, -3.2, -2.3, -1.4, 
#         -0.5, 0.4, 1.3, 2.2, 3.1, 4.]
# )

# xe_arr = np.array([0])
# xe_arr = 0.5 + 0.5*np.tanh(
#     [-5., -4.1, -3.2, -2.3, 
#      -1.4, -0.8, -0.5, -0.24, 
#      0.0, 0.2, 0.4, 0.7, 
#      1.3, 2.2, 3.1, 4.]
# )
# xe_arr = np.array([.99,.999])

xe_arr = 0.5 + 0.5*np.tanh(
    [-5., -4.1, -3.2, -2.3, 
     -1.8, -1.4, -1, -0.8, -0.65, -0.5, -0.24, 
     0.0, 0.2, 0.4, 0.7, 
     1.3, 2.2, 3.1, 4.]
)

rs_arr    = info_list[0,0,:,1]
mwimp_arr = info_list[0,:,0,0]
photeng = abscissae_list[0,0,0,0,:]
eleceng = abscissae_list[0,0,0,1,:]

## Raw Data Manipulation

The transfer function is calculated by injecting 2 photons at the input energy abscissa. However, the first step is to assign $x$ photons to the top bin of the *output* photon abscissa, such that $x$ times the energy of the top bin is 2 times the injection energy. Because of the misalignment between the two abscissae, as well as the fact that we want to transfer function for a single electron, we have to normalize the results first.

First, we construct two arrays: a list of output abscissae (for both photons and electrons) given the injection abscissa, and a list of the energy of the *output* photon energy bin where the injected photons are assigned to. Then we compute the normalization factor. 

In [ ]:
# This treatment is consistent with ih_transferfunction
photeng_bin_width = np.diff(np.log(spectools.get_bin_bound(photeng))) * photeng
eleceng_bin_width = np.diff(np.log(spectools.get_bin_bound(eleceng))) * eleceng


# Original injection energies
top_photeng_bins = np.array(
    [
        photeng[photeng <= mwimp][-1]
        for mwimp in mwimp_arr
    ]
)

norm_fac_arr = 2*mwimp_arr/top_photeng_bins

Dividing by the normalization factor gives the spectra for the injection of 1 photon, with energy given by `top_photeng_bins`. 

Now we construct a raw array of all of the transfer functions, on which we will perform an interpolation to extract a set of transfer functions for use, given a chosen abscissa.

In [ ]:
# Multiplying by bin width converts all the eleceng array into number of particles. 
# Indexing: xe, in_eng, rs, eng, type={highengphot, lowengphot, lowengelec}

phot_tf_raw = np.array([
    [
        [
            raw_tf[i,j,:,0]/norm_fac*photeng_bin_width
                for (j, rs) in enumerate(rs_arr)   
        ]
        for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))
        
    ] for raw_tf in tqdm(raw_tf_list)
])

lowengphot_tf_raw = np.array([
    [
        [
            raw_tf[i,j,:,1]/norm_fac*photeng_bin_width 
                for (j, rs) in enumerate(rs_arr)   
        ]
        for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))
        
    ] for raw_tf in tqdm(raw_tf_list)
])

lowengelec_tf_raw = np.array([
    [
        [
            raw_tf[i,j,:,2]/norm_fac*eleceng_bin_width 
                for (j, rs) in enumerate(rs_arr)   
        ]
        for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))
        
    ] for raw_tf in tqdm(raw_tf_list)
])

# xe, in_eng, rs
CMB_engloss = CMB_engloss_list/np.outer(norm_fac_arr, np.ones_like(rs_arr))

# xe, in_eng, rs, {H-Ion, Exc, Heat, Cont}
denom_highdeposited = np.ones((xe_arr.size, rs_arr.size, 4, norm_fac_arr.size))
denom_highdeposited *= norm_fac_arr
denom_highdeposited = np.rollaxis(denom_highdeposited, 3, 1)
highdeposited = highdeposited_list/denom_highdeposited

## Some Plots to Check

In [ ]:
plt.figure()

injE_ind = 180

#xe, in_eng, rs, eng
plt.plot(photeng, phot_tf_raw[-1, injE_ind , -1, :]/photeng_bin_width)

ax = plt.gca()

ax.set_xscale('log')
ax.set_yscale('log')

# These two should be similar: 
# It is the file total energy in the high energy photon spectrum
# compared to the injection energy. 
print(np.dot(photeng, phot_tf_raw[-1, injE_ind, -1, :]))
print(top_photeng_bins[injE_ind])

In [ ]:
photeng_high = photeng[photeng > 60]
print(photeng_high.shape)

new_phot_tf_raw = phot_tf_raw
new_lowengphot_tf_raw = lowengphot_tf_raw
new_lowengelec_tf_raw = lowengelec_tf_raw
new_cmbloss_raw = CMB_engloss
new_highdeposited_raw = highdeposited

Now, we construct the list of `Spectrum` objects from our raw array of transfer functions...

In [ ]:
# spec_type = 'N' for eleceng, because we have already multiplied in the bin width earlier. 

#xe, in_eng, rs, eng
photspec_list = [
    [
        [
            Spectrum(
                photeng, phot_tf[i,j,:], spec_type='N', 
                rs = rs, in_eng = in_eng
            ) for (j, rs) in enumerate(rs_arr)
        ] for (i, in_eng) in enumerate(photeng_high)
    ] for phot_tf in tqdm(new_phot_tf_raw)
]

lowengphotspec_list = [
    [
        [
            Spectrum(
                photeng, lowengphot_tf[i,j,:], spec_type='N', 
                rs = rs, in_eng = in_eng
            ) for (j, rs) in enumerate(rs_arr)
        ] for (i, in_eng) in enumerate(photeng_high)
    ] for lowengphot_tf in tqdm(new_lowengphot_tf_raw)
]

lowengelecspec_list = [
    [
        [
            Spectrum(
                eleceng, lowengelec_tf[i,j,:], spec_type='N', 
                rs = rs, in_eng = in_eng
            ) for (j, rs) in enumerate(rs_arr)
        ] for (i, in_eng) in enumerate(photeng_high)
    ] for lowengelec_tf in tqdm(new_lowengelec_tf_raw)
]


Finally, we can construct the `TransferFuncList` from this.

In [ ]:
tfunclist_photspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
            for spec_arr in photspec_xe
        ]
    ) for photspec_xe in tqdm(photspec_list)
]

tfunclist_lowengphotspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
            for spec_arr in lowengphotspec_xe
        ]
    ) for lowengphotspec_xe in tqdm(lowengphotspec_list)
]
    

tfunclist_lowengelecspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
            for spec_arr in lowengelecspec_xe
        ]
    ) for lowengelecspec_xe in tqdm(lowengelecspec_list)
]

We will now transpose all of the `TransferFuncList` objects so that they are now all of type `'rs'`.

In [ ]:
for tflist_hep, tflist_lep, tflist_lee in zip(
    tfunclist_photspec_list, tfunclist_lowengphotspec_list, tfunclist_lowengelecspec_list):
    tflist_hep.transpose()
    tflist_lep.transpose()
    tflist_lee.transpose()
    for i, tf_hep, tf_lep, tf_lee in zip(np.arange(327), tflist_hep, tflist_lep, tflist_lee):
        if np.any(np.isnan(tf_hep._grid_vals)):
            print(np.where(np.isnan(tf_hep._grid_vals)),'here!')
        if np.any(np.isnan(tf_lep._grid_vals)):
            print(np.where(np.isnan(tf_lep._grid_vals)),'here1!')
        if np.any(np.isnan(tf_lee._grid_vals)):
            print(np.where(np.isnan(tf_lee._grid_vals)),'here2!')
        
        tf_hep._grid_vals[np.where(tf_hep.grid_vals < 0)] = 1e-100
        tf_lep._grid_vals[np.where(tf_lep.grid_vals < 0)] = 1e-100
        tf_lee._grid_vals[np.where(tf_lee.grid_vals < 0)] = 1e-100

In [ ]:
tfunclist_photspec_list[0].rs

In [ ]:
test = tfunclist_photspec_list[-1][-1]
# print(test.rs)

plt.figure()

ax = plt.gca()

# print(test.in_eng[114])
test.plot(ax, step=5)

# test.plot(ax, ind=114, indtype='ind')

# test.plot(ax, ind=(20,25), step=1)
# test.plot(ax, ind=(48, 51), step=1)

ax.loglog()

# plt.axis([3e3, 1e5, 1e-26, 1e3])

In [ ]:
ind = 220
# print(test.in_eng)
print(test.in_eng[ind])
print(np.log10(test.in_eng[ind]))
print(np.dot(test[ind].N,test[ind].eng))
print(test[ind].toteng())
print(test.in_eng[ind]/test[ind].toteng())

In [ ]:
test = tfunclist_lowengphotspec_list[-1][-1]

plt.figure()

ax = plt.gca()

test.plot(ax, step=5)

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 5e12, 1e-42, 3e2])

In [ ]:
test = tfunclist_lowengelecspec_list[-1][-1]

plt.figure()

ax = plt.gca()

test.plot(ax,step=5)

# test.plot(ax, ind=(0, 326), step=1)
# test.plot(ax, ind=np.array([50]))
ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-1, 4e3, 1e-12, 1e2])

In [ ]:
tfunclist_lowengelecspec_list[-1][0].in_eng[76]

Save the transfer functions.

In [ ]:
len(tfunclist_photspec_list)

In [ ]:
def save_as_pickled_object(obj, filepath):
    """
    This is a defensive way to write pickle.write, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

In [ ]:
direc = "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/complete/"
direc = "/home/gridgway/Desktop/DH_Data/"

# for complete_coarse

# pickle.dump(tfunclist_photspec_list, 
#            open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/tfunclist_photspec_60eV_injE_complete_rs_30_xe_2pts.raw", "wb")
#            )

# pickle.dump(tfunclist_lowengphotspec_list, 
#            open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/tfunclist_lowengphotspec_60eV_injE_complete_rs_30_xe_2pts.raw", "wb")
#            )

# pickle.dump(tfunclist_lowengelecspec_list, 
#            open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/tfunclist_lowengelecspec_60eV_injE_complete_rs_30_xe_2pts.raw", "wb")
#            )

# pickle.dump(
#     new_cmbloss_raw, 
#     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/CMB_engloss_60eV_injE_complete_rs_30_xe_2pts.raw", "wb")       
#            )

# pickle.dump(
#     new_highdeposited_raw, 
#     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/highdeposited_60eV_injE_complete_rs_30_xe_2pts.raw", "wb")       
#            )

# pickle.dump(tfunclist_photspec_list, open(direc+"tfunclist_photspec_60eV_complete_post1700_xHe0.raw", "wb"))

save_as_pickled_object(tfunclist_photspec_list, "/home/gridgway/Desktop/DH_Data/tfunclist_photspec_60eV_complete_post1700_xHe0.raw")

# pickle.dump(tfunclist_lowengphotspec_list, 
#            open(direc+"tfunclist_lowengphotspec_60eV_complete_post1700_xHe0.raw", "wb")
#            )

save_as_pickled_object(tfunclist_lowengphotspec_list,"/home/gridgway/Desktop/DH_Data/tfunclist_lowengphotspec_60eV_complete_post1700_xHe0.raw")

# pickle.dump(tfunclist_lowengelecspec_list, 
#            open(direc+"tfunclist_lowengelecspec_60eV_complete_post1700_xHe0.raw", "wb")
#            )

save_as_pickled_object(tfunclist_lowengelecspec_list,"/home/gridgway/Desktop/DH_Data/tfunclist_lowengelecspec_60eV_complete_post1700_xHe0.raw")

# pickle.dump(
#     new_cmbloss_raw, 
#     open(direc+"CMB_engloss_60eV_complete_post1700_xHe0.raw", "wb")       
#            )

save_as_pickled_object(new_cmbloss_raw,"/home/gridgway/Desktop/DH_Data/CMB_engloss_60eV_complete_post1700_xHe0.raw")

# pickle.dump(
#     new_highdeposited_raw, 
#     open(direc+"highdeposited_60eV_complete_post1700_xHe0.raw", "wb")       
#            )

save_as_pickled_object(new_highdeposited_raw,"/home/gridgway/Desktop/DH_Data/highdeposited_60eV_complete_post1700_xHe0.raw")

########################################################
########################################################

# for z=31 test one xe.

# pickle.dump(tfunclist_photspec_list, 
#            open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_list_new_fixed_binning.raw", "wb")
#            )

# pickle.dump(tfunclist_lowengphotspec_list, 
#            open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_list_new_fixed_binning.raw", "wb")
#            )

# pickle.dump(tfunclist_lowengelecspec_list, 
#            open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_list_new_fixed_binning.raw", "wb")
#            )

# pickle.dump(
#     new_cmbloss_raw, 
#     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_list_new_fixed_binning.raw", "wb")       
#            )

In [ ]:
test = tfunclist_photspec_list[-9][0]
print(test.in_eng[12])
print(np.log10(test.in_eng[12]))
# print(test.rs)

plt.figure()
ax = plt.gca()

ax.loglog()

plt.axis([1e1, 1e3, 1e-10, 1])
plt.axvline(x=163.153)

for i in np.arange(5):
    tfunclist_photspec_list[i][0].plot(ax, ind=12, indtype='ind')